In [1]:
!pip install -q -U trl transformers accelerate
!pip install -q datasets
!pip install openai
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.0/118.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 92.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.4/422.4 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Mounting G-Drive to save and load files

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


# Loading Dataset
Loading glue, cola Dataset which contains English acceptability judgments drawn from books and journal articles on linguistic theory with label whether it is a grammatical English sentence(label 1) or not(label 0).


In [ ]:
from datasets import load_dataset

dataset = load_dataset("glue", "cola")
# Just take the training split as using T4
dataset = dataset["train"]

Generating train split:   0%|          | 0/8551 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1043 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1063 [00:00<?, ? examples/s]

# Fine-tune BERT

In [ ]:
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
tokenized_data = tokenizer(dataset["sentence"], return_tensors="np", padding=True)

# Tokenizer returns a BatchEncoding, but we convert that to a dict for Keras
tokenized_data = dict(tokenized_data)

labels = np.array(dataset["label"])

# Splitting data
train_data, test_data, train_labels, test_labels = train_test_split(tokenized_data["input_ids"], labels, test_size=0.2)

In [ ]:
from transformers import TFAutoModelForSequenceClassification
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

# Load and compile our model
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased")
model.compile(optimizer=Adam(3e-5), loss=SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'], run_eagerly=True)  # No loss argument!

model.fit(train_data, train_labels, epochs=3)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3


214/214 [==============================] - 319s 1s/step - loss: 0.5323 - accuracy: 0.7436
Epoch 2/3
214/214 [==============================] - 172s 803ms/step - loss: 0.3363 - accuracy: 0.8575
Epoch 3/3
214/214 [==============================] - 168s 786ms/step - loss: 0.1849 - accuracy: 0.9308


# Evaluation

In [ ]:
loss, accuracy = model.evaluate(test_data, test_labels)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy*100:.2f}%")

# Received Test accuracy as 82.70 percentage with an loss of 0.49

54/54 [==============================] - 12s 224ms/step - loss: 0.4914 - accuracy: 0.8270
Test Loss: 0.4914
Test Accuracy: 82.70%


# Save and Load Model

In [ ]:
import pickle

pickle.dump(model, open('/content/drive/MyDrive/Model_LLM_Finetune/model.pkl', 'wb'))

/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


In [ ]:
# model = pickle.load(open('/content/drive/MyDrive/Model_LLM_Finetune/model.pkl', 'rb'))

# Embeddings

In [ ]:
import tensorflow as tf

embeddings_list = []

# Function to Create Embeddings
def embed_gen(sentence):
    # Tokenize the sentences
    tokens = tokenizer(sentence, return_tensors="np", padding=True)

    # Get embeddings for each token in a sentence
    output = model(**tokens, output_hidden_states=True)
    embedding = output.hidden_states[-1]

    # Average the embeddings along the sequence dimension to represent the whole sentence.
    avg_embedding1 = tf.reduce_mean(embedding, axis=1)

    # storing the embedding in a list
    embeddings_list.append(avg_embedding1[0].numpy().tolist())

In [ ]:
i = 0
embeddings_list = []

# Generate Embeddings
for sentence in dataset['sentence']:
    if (i%500) == 0:
        print(i)
    embed_gen(sentence)
    i += 1

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500


In [ ]:
import json

# Save Embeddings in a txt File
def save_list_to_txt(data_list, output_file):
    with open(output_file, 'w') as txt_file:
        for item in data_list:
            txt_file.write(str(item) + "\n")

output_file = "embeddings_final_.txt"
save_list_to_txt(embeddings_list, output_file)

# ChromaDB

In [ ]:
# Creating unique Ids corresponding to each sentence for ChromaDB Requirements

ids = []
i = 1
for i in range(len(dataset['sentence'])):
  ids.append(f"{i}")
  i += 1

In [ ]:
import chromadb

# create a local client.
# chroma_client = chromadb.Client()

# Initialize a persistent chroma client that saves to the given path.
chroma_client = chromadb.PersistentClient(path="/content/drive/MyDrive/Embeddings_Database_")

# load an existing collection
# collection = chroma_client.get_collection(name="Embeddings")

# delete an existing "Embeddings" collection.
# chroma_client.delete_collection("Embeddings")

# Create a new collection named "Embeddings" in the database.
collection = chroma_client.create_collection(name="Embeddings")

# Store the embeddings into the created collection.
# - ids: Unique identifiers for each embedding.
# - documents: The corresponding sentences for each embedding.
# - embeddings: The embeddings that we generated would be stored.
collection.add(
    ids=ids,
    documents=dataset["sentence"],
    embeddings=embeddings_list
)

# Inference

Taking inferences over generated Embeds using ChromaDB and manually comparing them using cosine similarity.

In [ ]:
import tensorflow as tf

# Function to generate Embedding
def embed_gen_user(sentence):
    # Tokenize the sentences
    tokens = tokenizer(sentence, return_tensors="np", padding=True)

    # Get embeddings for each token in a sentence
    output = model(**tokens, output_hidden_states=True)
    embedding = output.hidden_states[-1]

    # Average the embeddings along the sequence dimension to represent the whole sentence.
    avg_embedding1 = tf.reduce_mean(embedding, axis=1)

    return avg_embedding1[0].numpy().tolist()

In [ ]:
# Few sentences from Dataset with label (1 for grammatical English sentence and 0 for not)
for i in range(25):
    print(f"Sentence : {dataset['sentence'][i]}, Label : {dataset['label'][i]}")

Sentence : Our friends won't buy this analysis, let alone the next one we propose., Label : 1
Sentence : One more pseudo generalization and I'm giving up., Label : 1
Sentence : One more pseudo generalization or I'm giving up., Label : 1
Sentence : The more we study verbs, the crazier they get., Label : 1
Sentence : Day by day the facts are getting murkier., Label : 1
Sentence : I'll fix you a drink., Label : 1
Sentence : Fred watered the plants flat., Label : 1
Sentence : Bill coughed his way out of the restaurant., Label : 1
Sentence : We're dancing the night away., Label : 1
Sentence : Herman hammered the metal flat., Label : 1
Sentence : The critics laughed the play off the stage., Label : 1
Sentence : The pond froze solid., Label : 1
Sentence : Bill rolled out of the room., Label : 1
Sentence : The gardener watered the flowers flat., Label : 1
Sentence : The gardener watered the flowers., Label : 1
Sentence : Bill broke the bathtub into pieces., Label : 1
Sentence : Bill broke the 

In [ ]:
def similar_sentence_cosine_sim(query):

    # Generate Embedding on user query
    user_embed = embed_gen_user(query)

    # Get Top three query with highest cosine similarity.
    max1 = 0.0
    queries = {}
    for embed in embeddings_list:
        cosine_similarity_value = -tf.keras.losses.cosine_similarity(embed, user_embed)
        if cosine_similarity_value.numpy() > 0.70 and cosine_similarity_value.numpy() > max1:
            max1 = cosine_similarity_value.numpy()
            index = embeddings_list.index(embed)
            query = dataset['sentence'][index]
            queries[cosine_similarity_value.numpy()] = query
    top_3 = sorted(queries.items(), key=lambda x: x[0], reverse=True)[:3]

    for accuracy, query in top_3:
        print(f"Query: {query}")

In [ ]:
# Example using cosine similarity

query = str(input("Enter the text to get similar sentence using cosine similarity : "))
similar_sentence_cosine_sim(query)

Enter the text to get similar sentence using cosine similarity : They are dancing happily
Query: They are dancing.
Query: She is dancing.
Query: The boys swim.


In [ ]:
# Example using chromaDB

query = str(input("Enter the text to get similar sentence using ChromaDB : "))
user_embed = embed_gen_user(query)
results = collection.query(query_embeddings = user_embed,n_results=3)
results

Enter the text to get similar sentence using ChromaDB : They are dancing happily


{'ids': [['5627', '5629', '5626']],
 'distances': [[8.493659019470215, 17.450851440429688, 20.508955001831055]],
 'metadatas': [[None, None, None]],
 'embeddings': None,
 'documents': [['They are dancing.',
   'The men are dancing.',
   'She is dancing.']]}

Both methods perform quite well, but I am prioritizing ChromaDB over manual comparison using cosine similarity. This is because ChromaDB is significantly faster than manual comparison.

# Generative AI

Sentence Analyzer using gpt-3.5-turbo-0613 through OpenAI

In [2]:
import openai

openai.api_key = "Private-Key"

In [ ]:
def sentence_analyzer_LLM(user_statement):

    user_embed = embed_gen_user(user_statement)
    results = collection.query(query_embeddings = user_embed, n_results=3)

    # print(results['documents'][0])

    similar_examples = []
    for document in results['documents'][0]:
        index = dataset['sentence'].index(document)
        similar_example = ""
        similar_example = f" statement : {document}; Label : {dataset['label'][index]}"
        # print(similar_example)
        similar_examples.append(similar_example)

    prompt = f"""Based on your expertise in linguistic acceptability, determine if the following statement is a grammatical English sentence:

    "{user_statement}"

    For reference, here are three similar examples from the Corpus of Linguistic Acceptability:

        1. "{similar_examples[0]}"
        2. "{similar_examples[1]}"
        3. "{similar_examples[2]}"

     Using these examples as a guide, and considering your knowledge, please provide the label for the user's statement. If it is a grammatical English sentence, label it as "Label : 1". If it is not, label it as "Label : 0".
    """

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": prompt}],
        temperature=1.0,
    )

    # print(f"Response :{response}")

    message = response["choices"][0]["message"]["content"]
    # print(f"Message : {message}")

    return message

In [5]:
query = str(input("Enter the sentence : "))
gpt_output = sentence_analyzer_LLM(query)
print(f"AI Generated Output : {gpt_output}")

Enter the sentence : They are dancing happily
AI Generated Output : Based on linguistic acceptability and the given examples, the statement "They are dancing happily" is a grammatical English sentence. Therefore, the label for this statement would be "Label: 1".


Here we are getting correct output using Gererative AI concept.

# Deployment (Locally)

In [ ]:
!pip install pyngrok
!pip install fastapi
!pip install nest-asyncio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=676359d8a78c2f6cf98192913568bc9ed1e0029561947ac5670f5b76eda55c03
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [ ]:
import uvicorn
from fastapi import FastAPI
import nest_asyncio
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel

In [ ]:
app = FastAPI()

In [ ]:
origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [ ]:
class model_input(BaseModel):

    query : str

In [ ]:
@app.post('/query')
def index(input_parameters : model_input):
    input_data = input_parameters.json()
    input_dictionary = json.loads(input_data)

    user_statement = input_dictionary['query']
    user_embed = embed_gen_user(user_statement)

    results = collection.query(query_embeddings = user_embed, n_results = 3)

    print(results['documents'][0])

    similar_examples = []
    for document in results['documents'][0]:
        index = dataset['sentence'].index(document)
        similar_example = ""
        similar_example = f" statement : {document}; Label : {dataset['label'][index]}"
        print(similar_example)
        similar_examples.append(similar_example)

    prompt = f"""You have extensive expertise in sentiment analysis. Please analyze the sentiment of the following statement:

    "{user_statement}"

    To make your analysis more accurate, I'm providing you with five similar examples based on the user's query:

    1. "{similar_examples[0]}"
    2. "{similar_examples[1]}"
    3. "{similar_examples[2]}"
    4. "{similar_examples[3]}"
    5. "{similar_examples[4]}"

    Please consider these examples as references and provide your assessment of whether the user's statement has a positive (label 1) or negative (label 0) sentiment"""

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": prompt}],
        temperature=1.0,
    )

    print(f"Response :{response}")

    message = response["choices"][0]["message"]["content"]
    print(f"Message : {message}")

    return {f"Message : {message}"}

In [ ]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)